In [ ]:
import json
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List

from bokeh.palettes import Category20
cat20 = Category20[20]

In [ ]:
processed_data_population_pkl_path = "C:\\Users\\dosre\\dev\\thesis-data\\experimentation\\output_data\\processed_data_population.pkl"
output_plot_root_dir_path = "C:\\Users\\dosre\\dev\\thesis-data\\experimentation\\experimentation_analysis"
os.makedirs(output_plot_root_dir_path, exist_ok=True)

In [ ]:
processed_data_population = pd.read_pickle(processed_data_population_pkl_path)

In [ ]:
matrices = processed_data_population["matrix"].unique()

In [ ]:
matrix = matrices[0]

matrix_plot_dir_path = os.path.join(output_plot_root_dir_path, matrix)
os.makedirs(matrix_plot_dir_path, exist_ok=True)

matrix_data_population = processed_data_population.loc[processed_data_population["matrix"] == matrix]

setups = matrix_data_population["setup"].unique()

In [ ]:
setup = setups[0]

setup_plot_dir_path = os.path.join(matrix_plot_dir_path, setup)
os.makedirs(setup_plot_dir_path, exist_ok=True)

matrix_setup_data_population = matrix_data_population.loc[matrix_data_population["setup"] == setup]

inner_iters = matrix_setup_data_population["inner_iter"].unique()

In [ ]:
inner_iter = inner_iters[0]

convergence_data_to_plot = matrix_setup_data_population.loc[matrix_setup_data_population["inner_iter"] == inner_iter]

In [ ]:
class SolverFormatInfo:

    fp_ids = {"FP64", "FP32", "FP16"}
    
    vp_ids = {
        "OuterRestartCount",
        "RelativeResidualThreshold",
        "CheckStagnation",
        "ThresholdToStagnation"
    }

    solver_color_fmt_dict = {
        "FP64": (cat20[0], ",-"),
        "FP32": (cat20[2], ",-"),
        "FP16": (cat20[4], ",-"),
        "OuterRestartCount": (cat20[6], ",-"),
        "RelativeResidualThreshold": (cat20[8], ",-"),
        "CheckStagnation": (cat20[10], ",-"),
        "ThresholdToStagnation": (cat20[12], ",-"),
        "SD_OuterRestartCount": (cat20[14], ",-"),
        "SD_RelativeResidualThreshold": (cat20[16], ",-"),
        "SD_CheckStagnation": (cat20[18], ",-"),
    }

class Convergence_Experiment:

    nan_r = r"(-nan|nan|-inf|inf)"

    def _replace_nan(self, s: str) -> str:
        return re.sub(self.nan_r, "NaN", s)

    def __init__(self, solver: str, file_paths: List[str]):

        self.all_initiated = True
        self.all_converged = True
        self.all_terminated = True

        self.convergence_data = []

        elapsed_times_ms = []
        inner_iteration_counts = []
        final_relress = []
        for file_path in file_paths:

            with open(file_path, "r") as file_in:
        
                file_data = json.loads(self._replace_nan(file_in.read()))

                self.id = file_data["id"]

                self.all_initiated = (self.all_initiated and (file_data["initiated"] == "true"))
                self.all_converged = (self.all_converged and (file_data["converged"] == "true"))
                self.all_terminated = (self.all_terminated and (file_data["terminated"] == "true"))

                elapsed_times_ms.append(file_data["elapsed_time_ms"])
                inner_iteration_counts.append(np.sum(file_data["inner_iterations"]))

                initial_res_norm = file_data["outer_res_norm_history"][0]
                assert file_data["inner_res_norm_history"][0][0] == initial_res_norm
                final_relress.append(file_data["outer_res_norm_history"][-1]/initial_res_norm)
                self.convergence_data.append(
                    np.array(file_data["inner_res_norm_history"]).flatten("C")/initial_res_norm
                )

        self.total_elapsed_time_ms = np.sum(elapsed_times_ms)
        self.average_elapsed_time_ms = np.mean(elapsed_times_ms)
        self.median_elapsed_time_ms = np.median(elapsed_times_ms)

        self.average_inner_iteration_count = np.mean(inner_iteration_counts)
        self.median_inner_iteration_count = np.median(inner_iteration_counts)

        self.average_final_relres = np.mean(final_relress)
        self.median_final_relres = np.median(final_relress)

    def plot_convergence_data(self, ax: plt.Axes):
        first = True
        for conv_data in self.convergence_data:
            if first:
                ax.plot(
                    conv_data,
                    SolverFormatInfo.solver_color_fmt_dict[self.id][1],
                    color=SolverFormatInfo.solver_color_fmt_dict[self.id][0],
                    label=self.id
                )
                first=False
            else:
                ax.plot(
                    conv_data,
                    SolverFormatInfo.solver_color_fmt_dict[self.id][1],
                    color=SolverFormatInfo.solver_color_fmt_dict[self.id][0]
                )
    

In [ ]:
conv_experiments = []
for row in convergence_data_to_plot.iterrows():
    row = row[1]
    conv_experiments.append(Convergence_Experiment(row.solver, row.file_paths))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4.8))

table_data = [["solver", "inner iterations", "relative time", "final relative residual", "converged"]]

fp64_base_elapsed_time_ms = 0.
for exp_data in conv_experiments:
    if exp_data.id == "FP64":
        fp64_base_elapsed_time_ms = exp_data.median_elapsed_time_ms

for exp_data in conv_experiments:
    exp_data.plot_convergence_data(ax)
    relative_time = exp_data.median_elapsed_time_ms/fp64_base_elapsed_time_ms
    table_data.append(
        [
            exp_data.id,
            int(exp_data.median_inner_iteration_count),
            np.nan if np.isnan(exp_data.median_final_relres) else f"{relative_time:.2g}",
            exp_data.median_final_relres,
            exp_data.all_converged
        ]
    )
ax.legend()
ax.set_xlabel("Inner Iteration")
ax.semilogy()
ax.grid()
ax.set_ylabel("$\\frac{|| b-Ax_{i}||_{2}}{||b-Ax_{0}||_{2}}$")

ax.table(table_data, loc="right")

fig.suptitle(f"{matrix} {setup} GMRES({inner_iter})")

# plt.savefig()
plt.show()